# Downloading Datasets


## NOAA Daily Observation Data
dataset overview:

https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C00861/html

main ftp directory:

ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/

readme:

ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt

required ftp files:
1. ghcnd-stations.txt
2. ghcnd-states.txt
3. ghcnd_ghc.tar.gz - YOU WILL NEED TO UNZIP THIS 


## Highway Rail Grade Crossing Accident Data
dataset overview:

https://data.transportation.gov/Railroads/Highway-Rail-Grade-Crossing-Accident-Data/7wn6-i5b9

download link:

https://data.transportation.gov/api/views/7wn6-i5b9/rows.csv?accessType=DOWNLOAD&bom=true&format=true

from the overview page, click export -> choose your output type (I chose CSV for this code).  or use the download link


## Weather Events 2016 - 2020
dataset overview:

https://www.kaggle.com/sobhanmoosavi/us-weather-events

download link:

https://www.kaggle.com/sobhanmoosavi/us-weather-events/download


## Major Safety Events

dataset overview:
- link missing 

dataset download:
- link missing
(we have it stored in drive but I dont know the original source)

https://drive.google.com/file/d/1eMuGXTA9W2YRJydmn6ff2MXZup7JJBfF/view?usp=sharing


# US Cities

dataset overview:

https://github.com/kelvins/US-Cities-Database

dataset download:

https://github.com/kelvins/US-Cities-Database/blob/main/csv/us_cities.csv

# imports

In [1]:
import pandas as pd
import numpy as np
import glob
import csv
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import datetime
import re

# NOAA Station Data Wrangling

In [2]:
'''
this dataset contains weather stations and their locations published by the noaa 
set is used to map daily weather observations against rail crossings
'''

# gather weather station metadata from the stations.txt file

# you will need to change the path to where your txt file resides
f = open("F:\weather\ghcnd-stations.txt","r")
lines = f.readlines()

# columns in the station file
colnames = ['ID', 'LAT', 'LON', 'ELEV', 'STATE', 'NAME', 'GSN', 'HCNCRN', 'WMOID']
stationlist = []

# initialize dataframe with correct columns
stations = pd.DataFrame(columns=colnames)

# iterate through stations and add them to our collection of stations if they are in the US
for line in lines:
    # first 2 characters are the country code , we only care about us stations
    if line[0:2] == 'US':
        
        # the description of the file seemed slightly off, i tested and found these column numbers to work best
        row = {"ID": line[0:11].upper(),
                "LAT": float(line[13:20]),
                "LON": float(line[21:30]),
                "ELEV": float(line[31:37]),
                "STATE": line[38:40],
                "NAME": line[41:71],
                "GSN": line[72:75],
                "HCNCRN": line[76:79],
                "WMOID": line[80:85]
               }
        stationlist.append(row)
    else:
        pass
stations = stations.append(stationlist)
f.close()


# read in states dataset to supplement weather stations data
fstates = open("F:\weather\ghcnd-states.txt","r")
lines3 = fstates.readlines()

colnames = ['CODE', 'NAME']

# create dataframe of state data
states = pd.DataFrame(columns=colnames)
for line in lines3:
    modline = line.strip('\n')
    data = {'CODE': line[0:2],
            "NAME": modline[3:50]
           }
    states = states.append(data, ignore_index=True)    

fstates.close()

# add state data to the stations dataset
stationplus = stations.join(states.set_index('CODE'), on='STATE', rsuffix='_STATE')

# create our key feature: coordinateID (wcoordinateID for weather)
# round latitude & longitude to 1 decimal, combine them in a tuple (lat, lon)
stationplus['wcoordinateID'] = list(zip(round(stationplus['LAT'],1),round(stationplus['LON'],1)))
stationplus = stationplus[['ID','ELEV','wcoordinateID']]

,ID,LAT,LON,ELEV,STATE,NAME,GSN,HCNCRN,WMOID,NAME_STATE,wcoordinateID
0,US009052008,43.7333,-96.6333,482.0,SD,SIOUX FALLS (ENVIRON. CANADA),,,,SOUTH DAKOTA,"(43.7, -96.6)"
1,US10RMHS145,40.5268,-105.1113,1569.1,CO,RMHS 1.6 SSW,,,,COLORADO,"(40.5, -105.1)"
2,US10ADAM001,40.5680,-98.5069,598.0,NE,JUNIATA 1.5 S,,,,NEBRASKA,"(40.6, -98.5)"
3,US10ADAM002,40.5093,-98.5493,601.1,NE,JUNIATA 6.0 SSW,,,,NEBRASKA,"(40.5, -98.5)"
4,US10ADAM003,40.4663,-98.6537,615.1,NE,HOLSTEIN 0.1 NW,,,,NEBRASKA,"(40.5, -98.7)"


# US Cities Data Wrangling

In [4]:
'''
this data will be used to attach coordinateID 
to other datasets that only have city or county level data
'''


cities = pd.read_csv('us_cities.csv')

# standardize county and state, city is not populated for all events.
# one change to approach would be to include all cities + the grouped mean of each county
cities['County'] = cities['COUNTY'].str.upper()
cities['State'] = cities['STATE_NAME'].str.upper()

# subset of data that we care about, lat+lon to make coordinateID, county, state, state code to merge on
counties = cities[['County','State','LATITUDE','LONGITUDE','STATE_CODE']]
grouped_counties = counties.groupby(['State','County'])
grouped_meancounties = grouped_counties.mean()
grouped_meancounties = grouped_meancounties.reset_index()
grouped_meancounties['wcoordinateID'] = list(zip(round(grouped_meancounties['LATITUDE'],1),round(grouped_meancounties['LONGITUDE'],1)))

# Rail Crossing Data Wrangling

In [22]:
'''
this dataset will be used to provide instances for our model to learn from.
we also use this dataset to limit the number of weather stations we include  observations from
'''

# ingest data
railcrossing = pd.read_csv('Highway-Rail_Grade_Crossing_Accident_Data.csv')


# gather the fields necessary for coordinateID, as well as any  fields you want for analysis later 
# change to approache what fields we include in refined_rr
refined_rr = railcrossing[['Incident Number','Date','County Name', 'State Name']]

# drop any incident without a date
refined_rr = refined_rr.dropna(subset=['Date'])

# create our feature incident date, which is an integer with format: yyyymmdd 
incident_date = refined_rr['Date'].str.split(' ', expand=True)
incident_date = incident_date[0].str.split('/', expand=True)
refined_rr['incident_date'] = (incident_date[2].astype(int) * 10000) + (incident_date[0].astype(int) * 100) + (incident_date[1].astype(int) * 1)

# merge accident data with city/county data to add coordinateID to each accident.
merg_rail_city = refined_rr.merge(grouped_meancounties, how='inner', left_on=['County Name','State Name'], right_on=['County','State'])
print(merg_rail_city.shape)
merg_rail_city = merg_rail_city[merg_rail_city['incident_date'] > 19900000]
merg_rail_city = merg_rail_city[['Incident Number', 'incident_date', 'wcoordinateID', 'State', 'County']]
print(merg_rail_city.shape)


(231891, 10)
(96411, 5)


In [23]:
merg_rail_city.head()

,Incident Number,incident_date,wcoordinateID,State,County
0,CA0109200,20090114,"(36.7, -119.7)",CALIFORNIA,FRESNO
1,CA0208200,20080209,"(36.7, -119.7)",CALIFORNIA,FRESNO
2,104019,20070404,"(36.7, -119.7)",CALIFORNIA,FRESNO
3,104074,20070410,"(36.7, -119.7)",CALIFORNIA,FRESNO
4,103616,20070221,"(36.7, -119.7)",CALIFORNIA,FRESNO


# Constricting Weather Station Inclusion

In [6]:
# dataframe of weather stations - only those that share coordinateID with an accident

merged_stations_accidents = stationplus.merge(merg_rail_city,left_on='wcoordinateID', right_on='wcoordinateID', how='inner')

# here we have a list of the station IDs that were included in the merged dataframe
incident_stations = [x.upper() for x in merged_stations_accidents['ID'].unique()]

# NOAA Daily Observations Data Wrangling

In [7]:
'''
prior attempts to include this data failed because ghcnd_ghc.tar.gz is too large.
by limiting the number of stations included to only those where incidents occurred,
and by limiting the observation years from each station, we can reduce the amount of
memory required to process this data

'''
# with assistance from 
# https://stackoverflow.com/questions/62165172/convert-dly-files-to-csv-using-python
# fields as given by the spec

fields = [
    ["ID", 1, 11],
    ["YEAR", 12, 15],
    ["MONTH", 16, 17],
    ["ELEMENT", 18, 21]]

offset = 22

for value in range(1, 32):
    fields.append((f"VALUE{value}", offset,     offset + 4))
    fields.append((f"MFLAG{value}", offset + 5, offset + 5))
    fields.append((f"QFLAG{value}", offset + 6, offset + 6))
    fields.append((f"SFLAG{value}", offset + 7, offset + 7))
    offset += 8

# Modify fields to use Python numbering
fields = [[var, start - 1, end] for var, start, end in fields]
fieldnames = [var for var, start, end in fields]

cwd = os.getcwd()

# the goal of this code is to make 1 file TOTAL from many (1 per station), which can then be filtered  by year. 

# enter where you want a csv saved - it will be many Gigs
csv_filename = cwd+'\\noaa_relevant_dailies.csv'

with open(csv_filename, 'w', newline='') as f_csv:
    
    # glob.glob should aim to where you unzipped the ghcnd_ghc.tar.gz , inside that will be ghcnd_hcn folder, which contains all the dailies
    for dly_filename in glob.glob(r'F:\weather\hcn\ghcnd_hcn\*.dly', recursive=True):
        path, name = os.path.split(dly_filename)
        station = name[:-4].upper()
        if station in incident_stations:
            # you could replace this with adding to a dataframe or something else, but i am running out of brain power.
            with open(dly_filename, newline='') as f_dly:
                spamwriter  = csv.writer(f_csv)
                spamwriter.writerow(fieldnames) 

                for line in f_dly:
                    row = [line[start:end].strip() for var, start, end in fields]
                    year = int(row[1])
                    
                    # important check to save memory, only add recent observations
                    if year > 1990:
                        spamwriter.writerow(row)

            
                
                
# end product is a csv with us weather station data.  needs more cleaning 

# NOAA Daily Data Wrangling Pt 2

In [8]:
df = pd.read_csv('noaa_relevant_dailies.csv')
# we added a header row for every file, but we only need 1 header row. remove the others:
df = df[df['YEAR'] != 'YEAR']

# month and year had some strings and some ints. lets standardize
df['YEAR'] = pd.to_numeric(df['YEAR'])
df['MONTH'] = pd.to_numeric(df['MONTH'])


# base for transposed data
base = pd.DataFrame(columns=['ID','YEAR','MONTH','ELEMENT','VALUE', 'MFLAG', 'QFLAG', 'SFLAG'])

# loop through all days to partially transpose the file (day cols -> rows)
for i in range(1,32):
    colnames = [f'VALUE{i}', f'MFLAG{i}', f'QFLAG{i}', f'SFLAG{i}']
    newcolnames = ['VALUE', 'MFLAG', 'QFLAG', 'SFLAG']
    col_order = ['ID','YEAR','MONTH','DAY','ELEMENT', colnames[0], colnames[1], colnames[2], colnames[3]]

    df_new = df[['ID','YEAR','MONTH','ELEMENT', colnames[0], colnames[1], colnames[2], colnames[3]]]
    df_new['DAY'] = i
    df_new = df_new[col_order]
    df_new = df_new.rename(columns={colnames[0]:newcolnames[0], colnames[1]:newcolnames[1], colnames[2]:newcolnames[2], colnames[3]:newcolnames[3]})
    base = pd.concat([base, df_new], sort=False)


newcsv = base[['ID','YEAR','MONTH','DAY','ELEMENT','VALUE','MFLAG','QFLAG','SFLAG']]

daily_station_coordinates = stationplus[['ID','wcoordinateID']]
daily_final = newcsv.merge(daily_station_coordinates, left_on='ID', right_on='ID')
daily_final.to_csv('final_daily_observations.csv')
daily_final.shape

C:\Users\Skurai\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,2,4,6,8,11,12,16,19,20,23,24,28,31,32,35,36,40,43,44,47,48,51,52,55,56,59,60,63,64,67,68,71,72,75,76,79,80,83,84,88,92,95,96,100,104,107,108,111,112,116,120,123,124,127) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Skurai\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(10880411, 10)

# Safety Events Data Wrangling

In [15]:
'''
not sure if we will use the safety events dataset or not
it is included here with incident date and coordinate ID so we can join on other datasets

'''

# ingest data
m_safety_events = pd.read_csv('Major_Safety_Events.csv')

# drop events without an area
m_safety_events.dropna(subset=['Primary UZA Name'],inplace=True)

# add city, state to each event
safety_events_citystate = m_safety_events['Primary UZA Name'].str.split(',', expand=True)
safety_events_state = safety_events_citystate[1].str.split('-', expand=True)
safety_events_state[0] = safety_events_state[0].str.strip()
m_safety_events['City'] = safety_events_citystate[0]
m_safety_events['State'] = safety_events_state[0]

# add coordinate ID by merging with cities data
m_safety_events = m_safety_events.merge(cities, how='inner', left_on=['City','State'], right_on=['CITY','STATE_CODE'])
m_safety_events['coordinateID'] = list(zip(round(m_safety_events['LATITUDE'],1),round(m_safety_events['LONGITUDE'],1)))

# add event date in integer format yyyymmdd
safety_events_date = m_safety_events['Incident Date'].str.split(' ', expand=True)
safety_events_date = safety_events_date[0].str.split('/', expand=True)
m_safety_events['event_date'] = (safety_events_date[2].astype(int) * 10000) + (safety_events_date[1].astype(int) * 100) + (safety_events_date[0].astype(int) * 1)

# filter for relevant events
inscope_events = ['Non-Rail Collision', 'Main Line Derailment', 'Rail Fire', 'Rail Collision', 'Flood','Ferry Boat Collision','Other High Winds','Tornado','Lightning','Hurricane']
inscope2_events = ['Non-Rail Collision', 'Main Line Derailment', 'Rail Fire', 'Rail Collision']

rail_events = m_safety_events[m_safety_events['Event Type'].isin(inscope_events)]
rail_events2 = m_safety_events[m_safety_events['Event Type'].isin(inscope2_events)]

print(m_safety_events.shape, rail_events.shape)
print(f'rail events represent about {round(100*(rail_events.shape[0]/m_safety_events.shape[0]),2)} percent of all events in the original data')

print(m_safety_events.shape, rail_events2.shape)
print(f'rail events2 represent about {round(100*(rail_events2.shape[0]/m_safety_events.shape[0]),2)} percent of all events in the original data')

# based on this data i recommend we use inscope2_events, if anything
 # can add more fields, but we need to be careful managing memory when we merge with other data
rail_events2 = rail_events2[['coordinateID', 'event_date', 'Incident Number', 'Event Type']]

(54007, 97) (43696, 97)
rail events represent about 80.91 percent of all events in the original data
(54007, 97) (43653, 97)
rail events2 represent about 80.83 percent of all events in the original data


# Weather Events Data Wrangling

In [10]:
'''
This data may not actually be used by us if we like the daily data
or possibly we will use both somehow
'''
w_events = pd.read_csv('WeatherEvents_Jan2016-Dec2020.csv')

# it looks like 'Severe' and 'Heavy' are most extreme, so filter to these 
extreme_severities = ['Severe', 'Heavy']
extreme_w_events = w_events[w_events['Severity'].isin(extreme_severities)]
print(w_events.shape, extreme_w_events.shape)
print(f'extreme events represent about {round(100*(extreme_w_events.shape[0]/w_events.shape[0]),2)} percent of all events in the original data')

# add coordinateID
extreme_w_events['wcoordinateID'] = list(zip(round(extreme_w_events['LocationLat'],1),round(extreme_w_events['LocationLng'],1)))

# calculate start and end dates as integers yyyymmdd
w_events_date_start = extreme_w_events['StartTime(UTC)'].str.split(' ', expand=True)
w_events_date_start = w_events_date_start[0].str.split('-', expand=True)
extreme_w_events['event_start_dt'] = (w_events_date_start[0].astype(int) * 10000) + (w_events_date_start[1].astype(int) * 100) + (w_events_date_start[2].astype(int) * 1)

w_events_date_end = extreme_w_events['EndTime(UTC)'].str.split(' ', expand=True)
w_events_date_end = w_events_date_end[0].str.split('-', expand=True)
extreme_w_events['event_end_dt'] = (w_events_date_end[0].astype(int) * 10000) + (w_events_date_end[1].astype(int) * 100) + (w_events_date_end[2].astype(int) * 1)



(6274206, 13) (1333526, 13)
extreme events represent about 21.25 percent of all events in the original data


C:\Users\Skurai\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Skurai\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Skurai\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

# Merging Safety Events on Weather Events

In [16]:
''' 
if we do include weather events data + safety events data
this is the code to merge them 
'''

# merge dataframes
safetyweather_merged = extreme_w_events.merge(rail_events2,left_on='wcoordinateID',right_on='coordinateID')

# number of days after event end date that we continue to look for incidents
lag_days = 2

# filter our dataframe for observations where weather event was near the safety event
final_safetyweather_merged = safetyweather_merged[(safetyweather_merged['event_date'] >= safetyweather_merged['event_start_dt']) & (safetyweather_merged['event_date'] <= safetyweather_merged['event_end_dt'] + lag_days)]



# Merging Rail Accidents on Weather Events

In [17]:
''' 
if we do include weather events data + railroad events data
this is the code to merge them 
'''

# merge dataframes
railweather_merged = extreme_w_events.merge(merg_rail_city,left_on='wcoordinateID',right_on='wcoordinateID')

# number of days after event end date that we continue to look for incidents
lag_days = 2

# filter our dataframe for observations where weather event was near the rail  event
final_railweather_merged = railweather_merged[(railweather_merged['incident_date'] >= railweather_merged['event_start_dt']) & (railweather_merged['incident_date'] <= railweather_merged['event_end_dt'] + lag_days)]


# Merging Rail Accidents on Daily Weather

In [24]:
'''
the most likely candidate for us to use for our model
'''

# memory error here, we have to reduce somewhere further up in the process....
raildaily_merged = daily_final.merge(merg_rail_city,left_on='wcoordinateID',right_on='wcoordinateID')

MemoryError: 